# ECS scene to GNN digestable format and scene classification example 

### Import libraries

In [2]:
from __future__ import annotations
import sys
sys.path.append("../../../")
import numpy as np
from torch_geometric.loader import DataLoader
import random
import torch
from torch.nn import Linear
from torch_geometric.nn import global_mean_pool, HeteroConv, SAGEConv
import Elements.pyECSS.utilities as util
from Elements.pyGLV.GL.GameObject import GameObject
import Converter
from Elements.pyECSS.Component import BasicTransform, RenderMesh
from Elements.pyECSS.Entity import Entity
from Elements.pyGLV.GL.Scene import Scene
from Elements.pyGLV.GL.Shader import Shader, ShaderGLDecorator
from Elements.pyGLV.GL.VertexArray import VertexArray
import Elements.pyGLV.utils.obj_to_mesh as obj

import numpy as np
import os

import Elements.pyECSS.utilities as util
from Elements.pyECSS.Entity import Entity
from Elements.pyECSS.Component import BasicTransform, RenderMesh, Camera
from Elements.pyECSS.System import TransformSystem, CameraSystem
from Elements.pyGLV.GL.Scene import Scene
from Elements.pyGLV.GUI.Viewer import RenderGLStateSystem

from Elements.pyGLV.GL.Shader import InitGLShaderSystem, Shader, ShaderGLDecorator, RenderGLShaderSystem
from Elements.pyGLV.GL.VertexArray import VertexArray
import Elements.pyGLV.utils.normals as norm

from OpenGL.GL import GL_LINES


### Function that creates a default living room scene.

In [3]:
def CreateRoomScene(visualize=False):
    scene = Scene()

    # Scenegraph with Entities, Components
    rootEntity = scene.world.createEntity(Entity(name="RooT"))
    entityCam1 = scene.world.createEntity(Entity(name="entityCam1"))
    scene.world.addEntityChild(rootEntity, entityCam1)
    trans1 = scene.world.addComponent(entityCam1, BasicTransform(name="trans1", trs=util.identity()))

    entityCam2 = scene.world.createEntity(Entity(name="entityCam2"))
    scene.world.addEntityChild(entityCam1, entityCam2)
    trans2 = scene.world.addComponent(entityCam2, BasicTransform(name="trans2", trs=util.identity()))
    orthoCam = scene.world.addComponent(entityCam2,
                                        Camera(util.ortho(-100.0, 100.0, -100.0, 100.0, 1.0, 100.0), "orthoCam",
                                               "Camera",
                                               "500"))

   

    # Colored Axes
    vertexAxes = np.array([
        [0.0, 0.0, 0.0, 1.0],
        [1.0, 0.0, 0.0, 1.0],
        [0.0, 0.0, 0.0, 1.0],
        [0.0, 1.0, 0.0, 1.0],
        [0.0, 0.0, 0.0, 1.0],
        [0.0, 0.0, 1.0, 1.0]
    ], dtype=np.float32)
    colorAxes = np.array([
        [1.0, 0.0, 0.0, 1.0],
        [1.0, 0.0, 0.0, 1.0],
        [0.0, 1.0, 0.0, 1.0],
        [0.0, 1.0, 0.0, 1.0],
        [0.0, 0.0, 1.0, 1.0],
        [0.0, 0.0, 1.0, 1.0]
    ], dtype=np.float32)

    # Simple Cube

    # index arrays for above vertex Arrays
    index = np.array((0, 1, 2), np.uint32)  # simple triangle
    indexAxes = np.array((0, 1, 2, 3, 4, 5), np.uint32)  # 3 simple colored Axes as R,G,B lines

    # Systems
    transUpdate = scene.world.createSystem(TransformSystem("transUpdate", "TransformSystem", "001"))
    camUpdate = scene.world.createSystem(CameraSystem("camUpdate", "CameraUpdate", "200"))
    renderUpdate = scene.world.createSystem(RenderGLShaderSystem())
    initUpdate = scene.world.createSystem(InitGLShaderSystem())
    shaders = []
    textureChair = "textures/dark_wood_texture.jpg"
    textureLamp = "models/LivingRoom/Lamp/10321_Table_Lamp_diffuse_v1.jpg"
    textureSofa = "models/LivingRoom/Sofa/10216_Bean_Bag_Chair_v1_Diffuse.jpg"
    textureSofa2 = "models/LivingRoom/Sofa2/10247_Sectional_Sofa_v1_Diffuse.jpg"
    textureTable = "models/LivingRoom/Table/Wood_Table_C_2.jpg"
    textureTV = "models/Scalpel/scalpel NEW 01B_LOW_Material _128_AlbedoTransparency.png"
    obj_to_import = "models/LivingRoom/Chair/Chair.obj"
    shaderchair = GameObject.Spawn(scene, obj_to_import, "Chair", rootEntity, util.translate(-0.2, 0, 0) @ util.scale(0.1) @ util.rotate((0, 1, 0), 0),
                                      textureChair)
    shaders.append(shaderchair)

    obj_to_import = "models/LivingRoom/Lamp/Lamp.obj"
    shaderlamp = GameObject.Spawn(scene, obj_to_import, "Lamp", rootEntity,  util.translate(-0.4, 0, 0.5) @ util.scale(0.1),
                                      textureLamp)
    shaders.append(shaderlamp)
    
    obj_to_import = "models/LivingRoom/Sofa/Sofa.obj"
    shadersofa = GameObject.Spawn(scene, obj_to_import, "Sofa", rootEntity, util.translate(0.2, 0, 0) @ util.scale(0.1) @ util.rotate((0, 1, 0), -90),
                                      textureSofa)
    shaders.append(shadersofa)

    obj_to_import = "models/LivingRoom/Sofa2/Sofa2.obj"
    shadersofa2 = GameObject.Spawn(scene, obj_to_import, "Sofa2", rootEntity,  util.translate(0, 0, 0.5) @ util.scale(0.1) @ util.rotate((0, 1, 0), 0),
                                      textureSofa2)
    shaders.append(shadersofa2)

   
    obj_to_import = "models/LivingRoom/Table/Table.obj"
    shadertable = GameObject.Spawn(scene, obj_to_import, "Table", rootEntity,  util.translate(0, 0, 0) @ util.scale(0.1),
                                      textureTable)
    shaders.append(shadertable)
    
    obj_to_import = "models/LivingRoom/TV/TV.obj"
    shaderTV = GameObject.Spawn(scene, obj_to_import, "TV", rootEntity, util.translate(-0.1, 0, -0.3) @ util.scale(0.1) @ util.rotate((0, 1, 0), 90),
                                      textureTV)
    shaders.append(shaderTV)
   

    # Generate terrain
    from Elements.pyGLV.utils.terrain import generateTerrain

    vertexTerrain, indexTerrain, colorTerrain = generateTerrain(size=4, N=20)
    # Add terrain
    terrain = scene.world.createEntity(Entity(name="terrain"))
    scene.world.addEntityChild(rootEntity, terrain)
    terrain_trans = scene.world.addComponent(terrain, BasicTransform(name="terrain_trans", trs=util.identity()))
    terrain_mesh = scene.world.addComponent(terrain, RenderMesh(name="terrain_mesh"))
    terrain_mesh.vertex_attributes.append(vertexTerrain)
    terrain_mesh.vertex_attributes.append(colorTerrain)
    terrain_mesh.vertex_index.append(indexTerrain)
    terrain_vArray = scene.world.addComponent(terrain, VertexArray(primitive=GL_LINES))
    terrain_shader = scene.world.addComponent(terrain, ShaderGLDecorator(
        Shader(vertex_source=Shader.COLOR_VERT_MVP, fragment_source=Shader.COLOR_FRAG)))
    # terrain_shader.setUniformVariable(key='modelViewProj', value=mvpMat, mat4=True)

    ## ADD AXES ##
    axes = scene.world.createEntity(Entity(name="axes"))
    scene.world.addEntityChild(rootEntity, axes)
    axes_trans = scene.world.addComponent(axes, BasicTransform(name="axes_trans", trs=util.identity()))
    axes_mesh = scene.world.addComponent(axes, RenderMesh(name="axes_mesh"))
    axes_mesh.vertex_attributes.append(vertexAxes)
    axes_mesh.vertex_attributes.append(colorAxes)
    axes_mesh.vertex_index.append(indexAxes)
    axes_vArray = scene.world.addComponent(axes, VertexArray(primitive=GL_LINES))  # note the primitive change

    # shaderDec_axes = scene.world.addComponent(axes, Shader())
    # OR
    axes_shader = scene.world.addComponent(axes, ShaderGLDecorator(
        Shader(vertex_source=Shader.COLOR_VERT_MVP, fragment_source=Shader.COLOR_FRAG)))
    # axes_shader.setUniformVariable(key='modelViewProj', value=mvpMat, mat4=True)

    # MAIN RENDERING LOOP

    running = True
    scene.init(imgui=True, windowWidth=1024, windowHeight=768, windowTitle="Elements: Textures example",
               openGLversion=4)

    # pre-pass scenegraph to initialise all GL context dependent geometry, shader classes
    # needs an active GL context
    scene.world.traverse_visit(initUpdate, scene.world.root)

    ################### EVENT MANAGER ###################

    eManager = scene.world.eventManager
    gWindow = scene.renderWindow
    gGUI = scene.gContext

    renderGLEventActuator = RenderGLStateSystem()
    transUpdate = scene.world.createSystem(TransformSystem("transUpdate", "TransformSystem", "001"))
    camUpdate = scene.world.createSystem(CameraSystem("camUpdate", "CameraUpdate", "200"))
    renderUpdate = scene.world.createSystem(RenderGLShaderSystem())
    initUpdate = scene.world.createSystem(InitGLShaderSystem())
    eManager._subscribers['OnUpdateWireframe'] = gWindow
    eManager._actuators['OnUpdateWireframe'] = renderGLEventActuator
    eManager._subscribers['OnUpdateCamera'] = gWindow
    eManager._actuators['OnUpdateCamera'] = renderGLEventActuator

    eManager._subscribers['OnUpdateWireframe'] = gWindow
    eManager._actuators['OnUpdateWireframe'] = renderGLEventActuator

    # START
    eManager._subscribers['OnUpdateCamera'] = gWindow
    eManager._actuators['OnUpdateCamera'] = renderGLEventActuator
    # END



    eye = util.vec(1.5, 1.5, 1.5)
    target = util.vec(0.0, 0.0, 0.0)
    up = util.vec(0.0, 1.0, 0.0)
    view = util.lookat(eye, target, up)
    # projMat = util.ortho(-10.0, 10.0, -10.0, 10.0, -1.0, 10.0)
    # projMat = util.perspective(90.0, 1.33, 0.1, 100)
    projMat = util.perspective(50.0, 1.0, 0.01, 10.0)

    

    gWindow._myCamera = view  # otherwise, an imgui slider must be moved to properly update

    

    model_terrain_axes = terrain.getChild(0).trs  # notice that terrain.getChild(0) == terrain_trans

    Lposition = util.vec(-1, 1.5, 1.2)  # uniform lightpos
    Lambientcolor = util.vec(1.0, 1.0, 1.0)  # uniform ambient color
    Lcolor = util.vec(1.0, 1.0, 1.0)
    Lintensity = 40.0
    for shader in shaders:
        shader.initialize_gl(Lposition, Lcolor, Lintensity)
   
    if visualize:
        while running:
            running = scene.render()
            scene.world.traverse_visit(renderUpdate, scene.world.root)
            scene.world.traverse_visit_pre_camera(camUpdate, orthoCam)
            scene.world.traverse_visit(camUpdate, scene.world.root)
            view = gWindow._myCamera  # updates view via the imgui
            mvp_terrain_axes = projMat @ view @ model_terrain_axes
            axes_shader.setUniformVariable(key='modelViewProj', value=mvp_terrain_axes, mat4=True)
            terrain_shader.setUniformVariable(key='modelViewProj', value=mvp_terrain_axes, mat4=True)
            for shader in shaders:
                model_cube = shader.transform_component.trs
                # model_cube = util.translate(0, 0, 0)
                shader.mesh_entities[0].shader_decorator_component.setUniformVariable(key='model', value=model_cube, mat4=True)
                shader.mesh_entities[0].shader_decorator_component.setUniformVariable(key='view', value=view, mat4=True)
                shader.mesh_entities[0].shader_decorator_component.setUniformVariable(key='projection', value=projMat, mat4=True)
                normalMatrix = np.transpose(util.inverse(model_cube))
                shader.mesh_entities[0].shader_decorator_component.setUniformVariable(key='normalMatrix', value=normalMatrix, mat4=True)
                shader.mesh_entities[0].shader_decorator_component.setUniformVariable(key='camPos', value=eye, float3=True)
            scene.render_post()

    scene.shutdown()
    return scene

### Function that creates a default operating room (OR) scene.

In [4]:
def CreateORScene(visualize=False):
    scene = Scene()

    # Scenegraph with Entities, Components
    rootEntity = scene.world.createEntity(Entity(name="RooT"))
    entityCam1 = scene.world.createEntity(Entity(name="entityCam1"))
    scene.world.addEntityChild(rootEntity, entityCam1)
    trans1 = scene.world.addComponent(entityCam1, BasicTransform(name="trans1", trs=util.identity()))

    entityCam2 = scene.world.createEntity(Entity(name="entityCam2"))
    scene.world.addEntityChild(entityCam1, entityCam2)
    trans2 = scene.world.addComponent(entityCam2, BasicTransform(name="trans2", trs=util.identity()))
    orthoCam = scene.world.addComponent(entityCam2,
                                        Camera(util.ortho(-100.0, 100.0, -100.0, 100.0, 1.0, 100.0), "orthoCam",
                                               "Camera",
                                               "500"))

   
    # Colored Axes
    vertexAxes = np.array([
        [0.0, 0.0, 0.0, 1.0],
        [1.0, 0.0, 0.0, 1.0],
        [0.0, 0.0, 0.0, 1.0],
        [0.0, 1.0, 0.0, 1.0],
        [0.0, 0.0, 0.0, 1.0],
        [0.0, 0.0, 1.0, 1.0]
    ], dtype=np.float32)
    colorAxes = np.array([
        [1.0, 0.0, 0.0, 1.0],
        [1.0, 0.0, 0.0, 1.0],
        [0.0, 1.0, 0.0, 1.0],
        [0.0, 1.0, 0.0, 1.0],
        [0.0, 0.0, 1.0, 1.0],
        [0.0, 0.0, 1.0, 1.0]
    ], dtype=np.float32)

    # Simple Cube

    # index arrays for above vertex Arrays
    index = np.array((0, 1, 2), np.uint32)  # simple triangle
    indexAxes = np.array((0, 1, 2, 3, 4, 5), np.uint32)  # 3 simple colored Axes as R,G,B lines

    # Systems
    transUpdate = scene.world.createSystem(TransformSystem("transUpdate", "TransformSystem", "001"))
    camUpdate = scene.world.createSystem(CameraSystem("camUpdate", "CameraUpdate", "200"))
    renderUpdate = scene.world.createSystem(RenderGLShaderSystem())
    initUpdate = scene.world.createSystem(InitGLShaderSystem())
    textureToolsTable = "models/ToolsTable/Cloth-TOOLtable_LOW_Material__126_AlbedoTransparency.png"
    textureScalpel = "models/Scalpel/scalpel NEW 01B_LOW_Material _128_AlbedoTransparency.png"
    textureCauterizer = "models/Cauterizer/cauterizer_low_01_Cauterizer_Blue_AlbedoTransparency.png"
    textureImplants = "models/ImplantsTable/table_with_implants_01_Material _3_AlbedoTransparency.png"
    textureAnesthesia = "models/Anesthesia/Anaisthesia UVS 02_Material _26_AlbedoTransparency.png"
    shaders = []
    obj_to_import = "models/ToolsTable/ToolsTable.obj"
    shaderToolsTable = GameObject.Spawn(scene, obj_to_import, "ToolsTable", rootEntity, util.translate(0, 0, 0),
                                      textureToolsTable)
    shaders.append(shaderToolsTable)

    obj_to_import = "models/Scalpel/Scalpel.obj"
    shaderscalpel = GameObject.Spawn(scene, obj_to_import, "Scalpel", rootEntity, util.translate(0, 0.5, 0),
                                      textureScalpel)
    shaders.append(shaderscalpel)
    
    obj_to_import = "models/Cauterizer/Cauterizer.obj"
    shadercauterizer = GameObject.Spawn(scene, obj_to_import, "Cauterizer", rootEntity, util.translate(0.3, 0.5, 0),
                                      textureCauterizer)
    shaders.append(shadercauterizer)

    obj_to_import = "models/ImplantsTable/ImplantsTable.obj"
    shaderimplants = GameObject.Spawn(scene, obj_to_import, "ImplantsTable", rootEntity,  util.translate(1, 0, 0),
                                      textureImplants)
    shaders.append(shaderimplants)

   
    obj_to_import = "models/Anesthesia/Anesthesia.obj"
    shaderanesthesia = GameObject.Spawn(scene, obj_to_import, "Anesthesia", rootEntity,  util.translate(-1, 0, 0) @ util.rotate((0, 1, 0), 0),
                                      textureAnesthesia)
    shaders.append(shaderanesthesia)
    


    # Generate terrain
    from Elements.pyGLV.utils.terrain import generateTerrain

    vertexTerrain, indexTerrain, colorTerrain = generateTerrain(size=4, N=20)
    # Add terrain
    terrain = scene.world.createEntity(Entity(name="terrain"))
    scene.world.addEntityChild(rootEntity, terrain)
    terrain_trans = scene.world.addComponent(terrain, BasicTransform(name="terrain_trans", trs=util.identity()))
    terrain_mesh = scene.world.addComponent(terrain, RenderMesh(name="terrain_mesh"))
    terrain_mesh.vertex_attributes.append(vertexTerrain)
    terrain_mesh.vertex_attributes.append(colorTerrain)
    terrain_mesh.vertex_index.append(indexTerrain)
    terrain_vArray = scene.world.addComponent(terrain, VertexArray(primitive=GL_LINES))
    terrain_shader = scene.world.addComponent(terrain, ShaderGLDecorator(
        Shader(vertex_source=Shader.COLOR_VERT_MVP, fragment_source=Shader.COLOR_FRAG)))
    # terrain_shader.setUniformVariable(key='modelViewProj', value=mvpMat, mat4=True)

    ## ADD AXES ##
    axes = scene.world.createEntity(Entity(name="axes"))
    scene.world.addEntityChild(rootEntity, axes)
    axes_trans = scene.world.addComponent(axes, BasicTransform(name="axes_trans", trs=util.identity()))
    axes_mesh = scene.world.addComponent(axes, RenderMesh(name="axes_mesh"))
    axes_mesh.vertex_attributes.append(vertexAxes)
    axes_mesh.vertex_attributes.append(colorAxes)
    axes_mesh.vertex_index.append(indexAxes)
    axes_vArray = scene.world.addComponent(axes, VertexArray(primitive=GL_LINES))  # note the primitive change

    # shaderDec_axes = scene.world.addComponent(axes, Shader())
    # OR
    axes_shader = scene.world.addComponent(axes, ShaderGLDecorator(
        Shader(vertex_source=Shader.COLOR_VERT_MVP, fragment_source=Shader.COLOR_FRAG)))
    # axes_shader.setUniformVariable(key='modelViewProj', value=mvpMat, mat4=True)

    # MAIN RENDERING LOOP

    running = True
    scene.init(imgui=True, windowWidth=1024, windowHeight=768, windowTitle="Elements: Textures example",
               openGLversion=4)

    # pre-pass scenegraph to initialise all GL context dependent geometry, shader classes
    # needs an active GL context
    scene.world.traverse_visit(initUpdate, scene.world.root)

    ################### EVENT MANAGER ###################

    eManager = scene.world.eventManager
    gWindow = scene.renderWindow
    gGUI = scene.gContext

    renderGLEventActuator = RenderGLStateSystem()
    transUpdate = scene.world.createSystem(TransformSystem("transUpdate", "TransformSystem", "001"))
    camUpdate = scene.world.createSystem(CameraSystem("camUpdate", "CameraUpdate", "200"))
    renderUpdate = scene.world.createSystem(RenderGLShaderSystem())
    initUpdate = scene.world.createSystem(InitGLShaderSystem())
    eManager._subscribers['OnUpdateWireframe'] = gWindow
    eManager._actuators['OnUpdateWireframe'] = renderGLEventActuator
    eManager._subscribers['OnUpdateCamera'] = gWindow
    eManager._actuators['OnUpdateCamera'] = renderGLEventActuator

    eManager._subscribers['OnUpdateWireframe'] = gWindow
    eManager._actuators['OnUpdateWireframe'] = renderGLEventActuator

    # START
    eManager._subscribers['OnUpdateCamera'] = gWindow
    eManager._actuators['OnUpdateCamera'] = renderGLEventActuator
    # END

    # Add RenderWindow to the EventManager publishers

    eye = util.vec(2.5, 2.5, 2.5)
    target = util.vec(0.0, 0.0, 0.0)
    up = util.vec(0.0, 1.0, 0.0)
    view = util.lookat(eye, target, up)

    projMat = util.perspective(50.0, 1.0, 0.01, 10.0)

    gWindow._myCamera = view  # otherwise, an imgui slider must be moved to properly update


    model_terrain_axes = terrain.getChild(0).trs  # notice that terrain.getChild(0) == terrain_trans
    Lposition = util.vec(-1, 1.5, 1.2)  # uniform lightpos
    Lambientcolor = util.vec(1.0, 1.0, 1.0)  # uniform ambient color
    Lcolor = util.vec(1.0, 1.0, 1.0)
    Lintensity = 40.0
    for shader in shaders:
        shader.initialize_gl(Lposition, Lcolor, Lintensity)
    if visualize:
        while running:
            running = scene.render()
            scene.world.traverse_visit(renderUpdate, scene.world.root)
            scene.world.traverse_visit_pre_camera(camUpdate, orthoCam)
            scene.world.traverse_visit(camUpdate, scene.world.root)
            view = gWindow._myCamera  # updates view via the imgui
            mvp_terrain_axes = projMat @ view @ model_terrain_axes
            axes_shader.setUniformVariable(key='modelViewProj', value=mvp_terrain_axes, mat4=True)
            terrain_shader.setUniformVariable(key='modelViewProj', value=mvp_terrain_axes, mat4=True)
            for shader in shaders:
                model_cube = shader.transform_component.trs
                # model_cube = util.translate(0, 0, 0)
                shader.mesh_entities[0].shader_decorator_component.setUniformVariable(key='model', value=model_cube, mat4=True)
                shader.mesh_entities[0].shader_decorator_component.setUniformVariable(key='view', value=view, mat4=True)
                shader.mesh_entities[0].shader_decorator_component.setUniformVariable(key='projection', value=projMat, mat4=True)
                normalMatrix = np.transpose(util.inverse(model_cube))
                shader.mesh_entities[0].shader_decorator_component.setUniformVariable(key='normalMatrix', value=normalMatrix, mat4=True)
                shader.mesh_entities[0].shader_decorator_component.setUniformVariable(key='camPos', value=eye, float3=True)
            scene.render_post()

    scene.shutdown()
    return scene

### Function that creates the OR scene as seen in the SIGGRAPH poster

In [5]:
insertPerf = True
removePerf = False
allShaders = []
USD_input_filepath = "PathToScene.usd"
def CreatePaperScene(visualize=False):
    import imgui
    from Elements.pyGLV.GUI.Viewer import ImGUIecssDecorator
    from Elements.pyGLV.GL.UsdImporter import LoadScene, SaveScene
    from Elements.pyGLV.utils.terrain import generateTerrain
    # from Elements.pyGLV.GL.Shader import UpdateUniformValuesSystem
    from Elements.pyGLV.GL.ActionSystems import InsertAction, InsertCollider, RemoveAction, RemoveComponent
    import OpenGL.GL as gl

    


    


    


    def SceneGUI(scene, initUpdate, textures):
        global USD_input_filepath
        global allShaders
        imgui.begin("Scene")

        changed, USD_input_filepath = imgui.input_text(label="filepath", buffer_length=400,
                                                   flags=imgui.INPUT_TEXT_ENTER_RETURNS_TRUE, value=USD_input_filepath)
        if imgui.button('Save Current Scene'):
            SaveScene(scene, USD_input_filepath)
            print('Scene was saved successfully.')

        if imgui.button('Load USD Scene'):
            allShaders = LoadScene(scene, USD_input_filepath)
            scene.world.traverse_visit(initUpdate, scene.world.root)
        imgui.end()
        return scene


    

    def CheckBoxGUI():
        global insertPerf
        global removePerf
        imgui.begin("Actions")
        # print(imgui.checkbox("Insert Action", insertPerf))
        if imgui.checkbox("Insert Action", insertPerf)[0]:
            insertPerf = not insertPerf
        if imgui.checkbox("Remove Action", removePerf)[0]:
            removePerf = not removePerf
        imgui.end()




    scene = Scene()

    # Scenegraph with Entities, Components
    rootEntity = scene.world.createEntity(Entity(name="RooT"))
    entityCam1 = scene.world.createEntity(Entity(name="Entity1"))
    scene.world.addEntityChild(rootEntity, entityCam1)

    eye = util.vec(1, 0.54, 1.0)
    target = util.vec(0.02, 0.14, 0.217)
    up = util.vec(0.0, 1.0, 0.0)
    view = util.lookat(eye, target, up)
    # projMat = util.ortho(-10.0, 10.0, -10.0, 10.0, -1.0, 10.0)
    # projMat = util.perspective(90.0, 1.33, 0.1, 100)
    projMat = util.perspective(50.0, 1.0, 1.0, 10.0)

    m = np.linalg.inv(projMat @ view)

    entityCam2 = scene.world.createEntity(Entity(name="Entity_Camera"))
    scene.world.addEntityChild(entityCam1, entityCam2)
    # orthoCam = scene.world.addComponent(entityCam2, Camera(util.ortho(-100.0, 100.0, -100.0, 100.0, 1.0, 100.0), "orthoCam","Camera","500"))
    orthoCam = scene.world.addComponent(entityCam2, Camera(m, "orthoCam", "Camera", "500"))

    # Colored Axes
    vertexAxes = np.array([
        [0.0, 0.0, 0.0, 1.0],
        [1.5, 0.0, 0.0, 1.0],
        [0.0, 0.0, 0.0, 1.0],
        [0.0, 1.5, 0.0, 1.0],
        [0.0, 0.0, 0.0, 1.0],
        [0.0, 0.0, 1.5, 1.0]
    ], dtype=np.float32)
    colorAxes = np.array([
        [1.0, 0.0, 0.0, 1.0],
        [1.0, 0.0, 0.0, 1.0],
        [0.0, 1.0, 0.0, 1.0],
        [0.0, 1.0, 0.0, 1.0],
        [0.0, 0.0, 1.0, 1.0],
        [0.0, 0.0, 1.0, 1.0]
    ], dtype=np.float32)

    # index arrays for above vertex Arrays
    indexAxes = np.array((0, 1, 2, 3, 4, 5), np.uint32)  # 3 simple colored Axes as R,G,B lines

    # Systems
    transUpdate = scene.world.createSystem(TransformSystem("transUpdate", "TransformSystem", "001"))
    camUpdate = scene.world.createSystem(CameraSystem("camUpdate", "CameraUpdate", "200"))
    renderUpdate = scene.world.createSystem(RenderGLShaderSystem())
    initUpdate = scene.world.createSystem(InitGLShaderSystem())
    # updateUniVals = scene.world.createSystem(UpdateUniformValuesSystem())
    
    vertexTerrain, indexTerrain, colorTerrain = generateTerrain(size=4, N=20)
    # Add terrain
    terrain = scene.world.createEntity(Entity(name="terrain"))
    scene.world.addEntityChild(rootEntity, terrain)
    terrain_trans = scene.world.addComponent(terrain, BasicTransform(name="terrain_trans",
                                                                    trs=util.identity() @ util.translate(0.0, -0.8, 0.0)))
    terrain_mesh = scene.world.addComponent(terrain, RenderMesh(name="terrain_mesh"))
    terrain_mesh.vertex_attributes.append(vertexTerrain)
    terrain_mesh.vertex_attributes.append(colorTerrain)
    terrain_mesh.vertex_index.append(indexTerrain)
    terrain_vArray = scene.world.addComponent(terrain, VertexArray(primitive=GL_LINES))
    terrain_shader = scene.world.addComponent(terrain, ShaderGLDecorator(
        Shader(vertex_source=Shader.COLOR_VERT_MVP, fragment_source=Shader.COLOR_FRAG)))
    # terrain_shader.setUniformVariable(key='modelViewProj', value=mvpMat, mat4=True)

    ## ADD AXES ##
    axes = scene.world.createEntity(Entity(name="axes"))
    scene.world.addEntityChild(rootEntity, axes)
    axes_trans = scene.world.addComponent(axes, BasicTransform(name="axes_trans",
                                                            trs=util.translate(0.0, 0.001, 0.0)))  # util.identity()
    axes_mesh = scene.world.addComponent(axes, RenderMesh(name="axes_mesh"))
    axes_mesh.vertex_attributes.append(vertexAxes)
    axes_mesh.vertex_attributes.append(colorAxes)
    axes_mesh.vertex_index.append(indexAxes)
    scene.world.addComponent(axes, VertexArray(primitive=gl.GL_LINES))  # note the primitive change

    # shaderDec_axes = scene.world.addComponent(axes, Shader())
    # OR
    axes_shader = scene.world.addComponent(axes, ShaderGLDecorator(
        Shader(vertex_source=Shader.COLOR_VERT_MVP, fragment_source=Shader.COLOR_FRAG)))
    # axes_shader.setUniformVariable(key='modelViewProj', value=mvpMat, mat4=True)

    running = True
    scene.init(imgui=True, windowWidth=1200, windowHeight=800, windowTitle="Elements: Tea anyone?", openGLversion=4,
            customImGUIdecorator=ImGUIecssDecorator)

    # pre-pass scenegraph to initialise all GL context dependent geometry, shader classes
    # needs an active GL context
    scene.world.traverse_visit(initUpdate, scene.world.root)

    ################### EVENT MANAGER ###################

    eManager = scene.world.eventManager
    gWindow = scene.renderWindow

    renderGLEventActuator = RenderGLStateSystem()

    eManager._subscribers['OnUpdateWireframe'] = gWindow
    eManager._actuators['OnUpdateWireframe'] = renderGLEventActuator
    eManager._subscribers['OnUpdateCamera'] = gWindow
    eManager._actuators['OnUpdateCamera'] = renderGLEventActuator

    eye = util.vec(2.5, 2.5, 2.5)
    target = util.vec(0.0, 0.0, 0.0)
    up = util.vec(0.0, 1.0, 0.0)
    view = util.lookat(eye, target, up)

    projMat = util.perspective(50.0, 1200 / 800, 0.01, 100.0)

    gWindow._myCamera = view  # otherwise, an imgui slider must be moved to properly update

    # ----Behavior setup-----
    # ------INSERT ACTION------
    insertColliderComponent = InsertCollider("insertCollider", "InsertCollider", 45, GameObject.Find("Hand"))
    scene.world.addComponent(GameObject.Find("BagMask"), insertColliderComponent)
    insertAction = InsertAction("insertAction", "InsertAction", "003")

    # ------Remove ACTION------
    removeComponent = RemoveComponent("removeComponent", "RemoveComponent", 0.2)
    scene.world.addComponent(GameObject.Find("Swab"), removeComponent)
    removeAction = RemoveAction("removeAction", "RemoveAction", "004")
    # ----------------------
    textures = []
    texture = "../models/ToolsTable/Cloth-TOOLtable_LOW_Material__126_AlbedoTransparency.png"
    textures.append(texture)
    texture = "../models/Scalpel/scalpel NEW 01B_LOW_Material _128_AlbedoTransparency.png"
    textures.append(texture)
    texture = "../models/Cauterizer/cauterizer_low_01_Cauterizer_Blue_AlbedoTransparency.png"
    textures.append(texture)
    texture = "../models/Anesthesia/Anaisthesia UVS 02_Material _26_AlbedoTransparency.png"
    textures.append(texture)
    texture = "../models/ImplantsTable/table_with_implants_01_Material _3_AlbedoTransparency.png"
    textures.append(texture)
    texture = "../models/Tray/TrayTexture.png"
    textures.append(texture)
    texture = "../models/Swab/Pliers.png"
    textures.append(texture)
    texture = "../models/BagMask/BagMaskTexture.png"
    textures.append(texture)
    texture = "../models/Hand/HandTexture.png"
    textures.append(texture)
    texture = "../models/ToolsTable/Cloth-TOOLtable_LOW_Material__126_AlbedoTransparency.png"
    textures.append(texture)
    Lposition = util.vec(-1, 1.5, 1.2)  # uniform lightpos
    Lambientcolor = util.vec(1.0, 1.0, 1.0)  # uniform ambient color
    Lcolor = util.vec(1.0, 1.0, 1.0)
    Lintensity = 40.0
    if visualize:
        while running:
            running = scene.render(running)
            scene.world.traverse_visit(renderUpdate, scene.world.root)
            scene.world.traverse_visit_pre_camera(camUpdate, orthoCam)
            scene.world.traverse_visit(camUpdate, scene.world.root)
            SceneGUI(scene, initUpdate, textures=textures)
            view = gWindow._myCamera  # updates view via the imgui
            # mvp_cube = projMat @ view @ model_cube
            # light_shader_decorator.setUniformVariable(key="modelViewProj", value=projMat @ view @ (
                    # util.translate(Lposition[0], Lposition[1], Lposition[2]) @ util.scale(1, 1, 1)), mat4=True)
            mvp_terrain = projMat @ view @ terrain_trans.trs
            mvp_axes = projMat @ view @ axes_trans.trs
            axes_shader.setUniformVariable(key='modelViewProj', value=mvp_axes, mat4=True)
            terrain_shader.setUniformVariable(key='modelViewProj', value=mvp_terrain, mat4=True)


            # Set Object Real Time Shader Data
            for shader in allShaders:
                model_cube = shader.parent.getChildByType(BasicTransform.getClassName()).trs
                # --- Set vertex shader data ---
                shader.setUniformVariable(key='projection', value=projMat, mat4=True)
                shader.setUniformVariable(key='view', value=view, mat4=True)
                shader.setUniformVariable(key='model', value=model_cube, mat4=True)

                # --- Set fragment shader data ---
                normalMatrix = np.transpose(util.inverse(model_cube))
                shader.setUniformVariable(key='normalMatrix', value=normalMatrix, mat4=True)

                shader.setUniformVariable(key='lightPos', value=Lposition, float3=True)
                shader.setUniformVariable(key='lightColor', value=Lcolor, float3=True)
                shader.setUniformVariable(key='lightIntensity', value=Lintensity, float1=True)
                # Camera position
                shader.setUniformVariable(key='camPos', value=eye, float3=True)
            scene.render_post()
    scene.shutdown()


### Visualize all 3 default scenes

In [ ]:
Scene.reset_instance()
defaultPaperScene = CreatePaperScene(visualize=True)
Scene.reset_instance()
defaultRoomScene = CreateRoomScene(visualize=True)
Scene.reset_instance()
defaultORScene = CreateORScene(visualize=True)
Scene.reset_instance()


Creating Scene Singleton Object
Creating Scene Singleton Object
SDL2Window: init()
Using OpenGL version 4.1
OpenGL 4.1.0 NVIDIA 526.56 GLSL 4.10 NVIDIA via Cg compiler Renderer NVIDIA GeForce RTX 3060 Laptop GPU/PCIe/SSE2
Entity  Hand  not found!
Entity  BagMask  not found!
Entity  Swab  not found!
Scene was loaded successfully.


AttributeError: 'str' object has no attribute '_texure_channel'

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\Users\dimit\miniconda3\envs\pyJANVRED\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "c:\Users\dimit\miniconda3\envs\pyJANVRED\lib\asyncio\selector_events.py", line 120, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\Users\dimit\miniconda3\envs\pyJANVRED\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "c:\Users\dimit\miniconda3\envs\pyJANVRED\lib\asyncio\selector_events.py", line 120, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetErro

: 

### Create the instances of the two default scenes, and convert them into a pytorch geometric graph format and also add the corresponding label to it

In [ ]:
Scene.reset_instance()
defaultRoomScene = CreateRoomScene()
defaultRoomSceneGNN = Converter.ECStoGNN(defaultRoomScene)
defaultRoomSceneGNN.y = 1
Scene.reset_instance()
defaultORScene = CreateORScene()
defaultORSceneGNN = Converter.ECStoGNN(defaultORScene)
defaultORSceneGNN.y = 0
Scene.reset_instance()
mydata = []
y = []

### Create 10 scenes, 5 of each room. The function ECStoGNN adds noise to each scene's component data so we get unique scenes each time. Finally we save them to a list

In [ ]:
numscenes = 100
for i in range(numscenes):
    print("I:", i)
    Scene.reset_instance()
    if i > numscenes / 2:
        y.append(0)
        scene = CreateORScene()
    else:
        y.append(1)
        scene = CreateRoomScene()

    data = Converter.ECStoGNN(scene)
    data.y = y[len(y) - 1]
    mydata.append(data)

mydata.append(defaultRoomSceneGNN)
mydata.append(defaultORSceneGNN)

### Create the GNN classifier's architecture. It consists of hetero convolutional layers that apply the SAGEConv operator on each type of edge. More information about the SAGEConv operator are here: 
https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.nn.conv.SAGEConv.html

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


## GNN Classifier architecture
class HeteroGNN(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels, num_layers):
        super().__init__()

        self.convs = torch.nn.ModuleList()
        for _ in range(num_layers):
            conv = HeteroConv({
                ('entity', 'entparent', 'entity'): SAGEConv((-1, -1), hidden_channels, normalize=True),
                ('entity', 'trsparent', 'trs'): SAGEConv((-1, -1), hidden_channels, normalize=True),
                ('entity', 'meshparent', 'mesh'): SAGEConv((-1, -1), hidden_channels, normalize=True),
            }, aggr='mean')
            self.convs.append(conv)

        self.lin1 = Linear(hidden_channels, out_channels)
        self.lin2 = Linear(hidden_channels, out_channels)
        self.lin3 = Linear(hidden_channels, out_channels)

    def forward(self, x_dict, edge_index_dict, batch1, batch2, batch3):
        for conv in self.convs:
            x_dict = conv(x_dict, edge_index_dict)
            x_dict = {key: x.relu() for key, x in x_dict.items()}
        x1 = global_mean_pool(x_dict['entity'], batch1)  # [batch_size, hidden_channels]
        x2 = global_mean_pool(x_dict['trs'], batch2)  # [batch_size, hidden_channels]
        x3 = global_mean_pool(x_dict['mesh'], batch3)  # [batch_size, hidden_channels]
        x1 = self.lin1(x1)
        x2 = self.lin2(x2)
        x3 = self.lin3(x3)
        final = x1 + x2 + x3
        final = torch.sigmoid(final)
        return final


model = HeteroGNN(hidden_channels=128, out_channels=2,
                  num_layers=5).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()
random.shuffle(mydata)

### Split the data into training and test set and train for 50 epochs. Calculate the loss and accuracy for each epoch on the test and training set. Finally, save the model

In [ ]:
train_data = mydata[:int(numscenes * 0.7)]
test_data = mydata[int(numscenes * 0.7):]
train_loader = DataLoader(train_data, batch_size=8, shuffle=True)
test_loader = DataLoader(test_data, batch_size=1, shuffle=True)

# Train for 50 epochs and print losses and accuracy
for i in range(50):
    correct = 0
    totalloss = 0
    model.train()
    for l in train_loader:
        a = model(l.x_dict, l.edge_index_dict, l['entity'].batch, l['trs'].batch, l['mesh'].batch)
        pred = a.argmax(dim=1).to(device)
       
        correct += int((pred == l.y.to(device)).sum())
        loss = criterion(a, l.y.to(device))  # Compute the loss.
        totalloss += loss.item()
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.
    model.eval()
    testcorrect = 0
    for l in test_loader:
        a = model(l.x_dict, l.edge_index_dict, l['entity'].batch, l['trs'].batch, l['mesh'].batch)
        pred = a.argmax(dim=1).to(device)
        # print(l.y.shape)
        testcorrect += int((pred == l.y.to(device)).sum())
    print(
        f'Epoch: {i:03d}, Train Acc: {correct / len(train_loader.dataset) :.4f}, Test Acc: {testcorrect / len(test_loader.dataset)} Loss: {totalloss / 8:.4f}')

torch.save(model.state_dict(), "scenemodel.pth")